# Reconciliation Monthly Stats

We need to load in these libraries into our notebook in order to query, load, manipulate and view the data

In [ ]:
import os
from datetime import datetime, timedelta
from config import Config
import requests
import pandas as pd

%load_ext sql
%config SqlMagic.displaylimit = 5

# Parameters cell for external parameters via papermill (job running this notebook will insert a parameter cell below this). This cell has a tag of with the name "parameters" that is used by papermill

e.g.
param1 = "some_value"

This will create the connection to the database and prep the jupyter magic for SQL

In [ ]:
%sql $Config.SQLALCHEMY_DATABASE_URI

Simplest query to run to ensure our libraries are loaded and our DB connection is working

In [ ]:
%%sql
set time zone 'UTC';

Query ...

In [ ]:
%%sql monthly_reconciliation_summary  <<
SELECT 
    id, 
    (created_on AT TIME ZONE 'UTC' AT TIME ZONE 'America/Vancouver')::date AS created_date,  
    total,  
    service_fees, 
    payment_method_code, 
    corp_type_code, 
    created_by, 
    payment_date
FROM 
    invoices
WHERE 
    corp_type_code = 'CSO'
    AND total > 0
    AND invoice_status_code = 'PAID'
    AND payment_method_code in ('PAD','EJV')
    AND created_on AT TIME ZONE 'UTC' AT TIME ZONE 'America/Vancouver'  > (current_date - 1 - interval '1 months' - interval '5 days')::date
    AND created_on AT TIME ZONE 'UTC' AT TIME ZONE 'America/Vancouver' <= (current_date - 1)::date
ORDER BY 
    1;

Save to CSV

In [ ]:
filename_summary = os.path.join(os.getcwd(), r'data/')+partner_code+'_monthly_reconciliation_summary_' + datetime.strftime(datetime.now()-timedelta(1), '%Y-%m') +'.csv'
df_summary = monthly_reconciliation_summary.DataFrame()
with open(filename_summary, 'w') as f:
    f.write('Monthly Reconciliation Summary:\n\n')
    if df_summary.empty:
        f.write('No Data Retrieved')
    else:
        df_summary.to_csv(f, sep=',', encoding='utf-8', index=False)

In [ ]:
%%sql monthly_reconciliation_disbursed  <<
SELECT id, (disbursement_date AT TIME ZONE 'UTC' AT TIME ZONE 'America/Vancouver')::date, total, service_fees, payment_method_code, corp_type_code,created_by
FROM invoices
WHERE corp_type_code = 'CSO'
AND invoice_status_code = 'PAID'
AND payment_method_code in ('PAD','EJV')
AND disbursement_status_code = 'COMPLETED'
AND disbursement_date AT TIME ZONE 'UTC' AT TIME ZONE 'America/Vancouver'  > (current_date - 1 - interval '1 months')::date
AND disbursement_date AT TIME ZONE 'UTC' AT TIME ZONE 'America/Vancouver'  <= (current_date - 1)::date
order by 1;

Save to another CSV

In [ ]:
filename_disbursed = os.path.join(os.getcwd(), r'data/')+partner_code+'_monthly_reconciliation_disbursed_' + datetime.strftime(datetime.now()-timedelta(1), '%Y-%m') +'.csv'
df_disbursed = monthly_reconciliation_disbursed.DataFrame()
with open(filename_disbursed, 'a') as f:
    f.write('Monthly Reconciliation Disbursed:\n\n')
    if df_disbursed.empty:
        f.write('No Data Retrieved')
    else:
        df_disbursed.to_csv(f, sep=',', encoding='utf-8', index=False)


In [ ]:
# Get the API base URL from the environment variable
API_BASE_URL = os.getenv('REPORT_API', '')
if not API_BASE_URL:
    raise ValueError("The REPORT_API environment variable is not set or is empty")

url = API_BASE_URL
headers = {
    'Authorization': 'Bearer your_jwt_token',
    'Content-Type': 'application/json',
    'Accept': 'application/pdf'
}

query = """
SELECT
    COUNT(*)                          AS transaction_count,
    pli.description,
    i.payment_method_code,
    'Courts Online' AS corp_type
FROM 
    invoices i
JOIN 
    payment_line_items pli ON i.id = pli.invoice_id
WHERE 
    i.corp_type_code = 'CSO'
    AND i.invoice_status_code = 'PAID'
    AND i.payment_method_code IN ('PAD', 'EJV')
    AND created_on AT TIME ZONE 'UTC' AT TIME ZONE 'America/Vancouver' >= '2023-05-01'
    AND created_on AT TIME ZONE 'UTC' AT TIME ZONE 'America/Vancouver' < '2023-07-01'
GROUP BY 
    pli.description,
    i.payment_method_code
ORDER BY 
    transaction_count
"""
result = %sql $query

# Convert the result to a DataFrame
df = result.DataFrame()

# Convert DataFrame to JSON-compatible format
query_results = df.to_dict(orient='records')
current_date = datetime.now().strftime("%B %d, %Y")
# Define the request body
data = {
    "templateVars": {
        "Date": current_date,
        "Company_Name": "LANCE LANDSCAPING LTD.",
        "Address_Line_1": "200 – 940 BLANSHARD STREET",
        "Address_Line_2": "",
        "City": "VICTORIA",
        "Province": "BC",
        "Area_Code": "V8W 3E6",
        "First_Name": "Lance Hall",
        "enter_month": "March",
        "invoice": {
            "_links": {
                "self": "http://auth-web.dev.com/api/v1/payment-requests/2",
                "collection": "http://auth-web.dev.com/api/v1/payment-requests?invoice_id=2"
            },
            "bcolAccount": "TEST",
            "corpTypeCode": "CP",
            "createdName": "test name",
            "id": 2,
            "createdBy": "test",
            "paymentAccount": {
                "accountId": "1234",
                "billable": True
            },
            "paymentDate": "2024-02-27T09:52:03+00:00",
            "total": 130.0,
            "paymentMethod": "CC",
            "overdueDate": "2024-03-01T09:52:02+00:00",
            "paid": 30.0,
            "details": [
                {
                    "label": "hello",
                    "value": "testing"
                }
            ],
            "serviceFees": 0.0,
            "updatedOn": "2024-02-27T09:52:03+00:00",
            "lineItems": [
                {
                    "waivedFees": None,
                    "waivedBy": None,
                    "gst": 0.0,
                    "pst": 0.0,
                    "filingFees": 10.0,
                    "id": 2,
                    "serviceFees": 0.0,
                    "priorityFees": 0.0,
                    "futureEffectiveFees": 0.0,
                    "quantity": 1,
                    "statusCode": "ACTIVE",
                    "total": 10.0,
                    "description": "NSF Charges"
                },
                {
                    "waivedFees": None,
                    "waivedBy": None,
                    "gst": 0.0,
                    "pst": 0.0,
                    "filingFees": 10.0,
                    "id": 1,
                    "serviceFees": 0.0,
                    "priorityFees": 0.0,
                    "futureEffectiveFees": 0.0,
                    "quantity": 1,
                    "statusCode": "ACTIVE",
                    "total": 10.0,
                    "description": "Good Product"
                }
            ],
            "createdOn": "2024-02-27T09:51:55+00:00",
            "references": [
                {
                    "invoiceNumber": "REG00001",
                    "id": 2,
                    "statusCode": "COMPLETED"
                }
            ],
            "receipts": [
                {
                    "receiptAmount": 100.0,
                    "id": 2,
                    "receiptDate": "2024-02-27T09:52:03.018524",
                    "receiptNumber": "1234567890"
                }
            ],
            "statusCode": "COMPLETED",
            "folioNumber": "1234567890"
        },
        "queryResults": query_results  # Include the query results in the request body
    },
    "templateName": "payment_receipt",
    "reportName": "payment_receipt"
}

# Send the POST request
response = requests.post(url, headers=headers, json=data)

# Check if the response is successful
if response.status_code == 200:
    # Get the PDF content from the response
    pdf_content = response.content
    
    # Save the PDF content to a file
    with open('payment_receipt.pdf', 'wb') as pdf_file:
        pdf_file.write(pdf_content)
    
    print("PDF report saved successfully as 'payment_receipt.pdf'")
else:
    print('Failed to get the report:', response.text)